In [1]:
from pathlib import Path
import sys  
import os
import pandas as pd 
from datetime import datetime
import scipy.sparse as sps
from numpy import linalg as LA

import numpy as np
from operator import itemgetter

In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "libs")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from Utils.load_URM import load_URM
from Utils.load_ICM import load_ICM

from scipy.sparse import hstack, vstack
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_all = load_URM("../data/data_train.csv")
ICM_all = load_ICM("../data/data_ICM_metadata.csv")

In [5]:
from libs.Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender 
from libs.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from libs.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from libs.Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

In [14]:
def AP(recommended_items, relevant_items):
   
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    
    # Cumulative sum: precision at 1, at 2, at 3 ...
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))
    ap_score = np.sum(p_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return ap_score

In [15]:
# We pass as paramether the recommender class

def evaluate_algorithm(URM_test, recommender_object, at=10):
    
    #cumulative_precision = 0.0
    #cumulative_recall = 0.0
    cumulative_AP = 0.0
    
    num_eval = 0


    for user_id in range(URM_test.shape[0]):

        relevant_items = URM_test.indices[URM_test.indptr[user_id]:URM_test.indptr[user_id+1]]
        
        if len(relevant_items)>0:
            
            recommended_items = recommender_object.recommend(user_id, cutoff=at)
            num_eval+=1

            #cumulative_precision += precision(recommended_items, relevant_items)
            #cumulative_recall += recall(recommended_items, relevant_items)
            cumulative_AP += AP(recommended_items, relevant_items)
            
    #cumulative_precision /= num_eval
    #cumulative_recall /= num_eval
    MAP = cumulative_AP / num_eval
    
    return MAP


In [13]:
URM_train = sps.load_npz('URM_train.npz')
URM_validation = sps.load_npz('URM_validation.npz')

In [7]:
# evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

In [8]:
slim_model_all = SLIMElasticNetRecommender(URM_all)
slim_model_all.load_model('result_experiments/SLIM/', 'slim_300_2_weights')

SLIMElasticNetRecommender: Loading model from file 'result_experiments/SLIM/slim_300_2_weights'
SLIMElasticNetRecommender: Loading complete


In [17]:
slim_model_buff = SLIMElasticNetRecommender(URM_all)
slim_model_buff.load_model('result_experiments/SLIM/', 'slim_300_2_weights')

SLIMElasticNetRecommender: Loading model from file 'result_experiments/SLIM/slim_300_2_weights'
SLIMElasticNetRecommender: Loading complete


In [18]:
(slim_model_all.W_sparse != slim_model_buff.W_sparse).nnz==0 

True

In [25]:
evaluate_algorithm(URM_validation, slim_model_all)

0.0

In [9]:
rp3_beta_all = RP3betaRecommender(URM_all)
rp3_beta_all.load_model('result_experiments/RP3beta/', 'rp3beta_10_weights')

RP3betaRecommender: Loading model from file 'result_experiments/RP3beta/rp3beta_10_weights'
RP3betaRecommender: Loading complete


In [29]:
evaluate_algorithm(URM_validation, rp3_beta_all)

0.0

In [10]:
ItemKNNCF_all = ItemKNNCFRecommender(URM_all)
ItemKNNCF_all.load_model("result_experiments/ItemKNNCFRecommender/", "ItemKNNCF_5_weights")

ItemKNNCFRecommender: Loading model from file 'result_experiments/ItemKNNCFRecommender/ItemKNNCF_5_weights'
ItemKNNCFRecommender: Loading complete


In [32]:
evaluate_algorithm(URM_validation, ItemKNNCF_all)

0.0

In [11]:
#new_similarity = 0.7963351700386568 * slim_model_all.W_sparse + 0.05677318666537923 * rp3_beta_all.W_sparse + 0.171828939868 * ItemKNNCF_all.W_sparse
#new_similarity = 0.00013801449258364696 * slim_model_all.W_sparse + 0.9998467429232237 * rp3_beta_all.W_sparse + 1.5242584192611179e-05 * ItemKNNCF_all.W_sparse
new_similarity = 0.48676074557629584 * slim_model_all.W_sparse + 0.5131741952410963 * rp3_beta_all.W_sparse + 6.505918260791844e-05 * ItemKNNCF_all.W_sparse
recommender_object_all = ItemKNNCustomSimilarityRecommender(URM_all)
recommender_object_all.fit(new_similarity)

In [34]:
recommender_object_all.save_model("result_experiments/ensemble_SLIM_RP3_KNNCFR/", "ensemble_weights")

ItemKNNCustomSimilarityRecommender: Saving model in file 'result_experiments/ensemble_SLIM_RP3_KNNCFR/ensemble_weights'
ItemKNNCustomSimilarityRecommender: Saving complete


In [35]:
# recommender_object_all = ItemKNNCustomSimilarityRecommender(URM_train)
# recommender_object_all.load_model("result_experiments/ensemble_SLIM_RP3_KNNCFR/", "ensemble_weights")

In [16]:
evaluate_algorithm(URM_validation, recommender_object_all)

0.0

In [17]:
def write_submission_lib_model(trained_model, filename: str = "submission.csv") -> None:
	"""Builds the submission file from a trained recommender model. The file is saved in a CSV format.

	:param trained_model: A fitted recommender model
	:type trained_model: RecommenderModel
	:param filename: The filename of the submission for this particular recommender model
	:type filename: str
	"""
	target_users_test = pd.read_csv("../data/data_target_users_test.csv",).to_numpy().ravel()

	recommendations = np.array([
		trained_model.recommend(user_id, cutoff=10) for user_id in target_users_test
	])

	if not os.path.exists("../submissions"):
		os.makedirs("../submissions")
	with open(f"../submissions/{filename}", "w") as f:
		f.write("user_id,item_list\n")
		for user_id, recs in zip(target_users_test, recommendations):
			f.write(f"{user_id},{' '.join(map(str, recs))}\n")

In [18]:
write_submission_lib_model(recommender_object_all, "ensemble_SLIM_RP3_KNNCF_5.csv")

In [19]:
recommender_object_all.recommend(100, cutoff=10)

[23253, 17532, 6822, 11300, 11307, 16350, 13279, 19055, 10468, 16327]

In [37]:
recommender_object_all = ItemKNNCustomSimilarityRecommender(URM_all)
recommender_object_all.load_model("result_experiments/ensemble_SLIM_RP3_KNNCFR/", "ensemble_weights")

ItemKNNCustomSimilarityRecommender: Loading model from file 'result_experiments/ensemble_SLIM_RP3_KNNCFR/ensemble_weights'
ItemKNNCustomSimilarityRecommender: Loading complete


In [38]:
recommender_object_all.recommend(100, cutoff=10)

[23253, 17532, 6822, 23215, 11307, 13279, 23225, 23190, 11300, 18217]

In [40]:
recommender_object_train = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object_train.load_model("result_experiments/ensemble_SLIM_RP3_KNNCFR/", "ensemble_weights")

ItemKNNCustomSimilarityRecommender: Loading model from file 'result_experiments/ensemble_SLIM_RP3_KNNCFR/ensemble_weights'
ItemKNNCustomSimilarityRecommender: Loading complete


In [41]:
recommender_object_train.recommend(100, cutoff=10)

[16310, 23253, 17532, 6822, 29025, 23215, 11307, 11300, 18217, 23277]